In [2]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd
from pytorch_lightning import Trainer
from pl_module import CMPNNLitModel, MultiCMPNNLitModel
from dataset import CMPNNDataset, MultiCMPNNDataset, MultiCMPNNDatasetSDF
from preprocessing import scaffold_split_indices, random_split_indices
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch
from torch.utils.data import Subset


In [5]:
targets_df.columns

Index(['rxn', 'r1_radius', 'r2_radius', 'alpha_angle', 'psi_1_dihedral',
       'psi_2_dihedral'],
      dtype='object')

In [4]:
root = "."
sdf_dir = "/home/calvin/code/chemprop_phd_customised/habnet/data/processed/sdf_data/"
targets = "/home/calvin/code/chemprop_phd_customised/habnet/data/processed/target_data/target_data.csv"
target_cols  = ["r1_radius", "r2_radius"]

targets_df = pd.read_csv(targets)
targets_df = targets_df[targets_df['label']== 'k_for (TST+T)'].reset_index(drop=True)

KeyError: 'label'

In [1]:
ds = MultiCMPNNDatasetSDF(
    root,
    sdf_files=sdf_dir,
    target_df=targets,
    target_cols=target_cols,
    atom_messages=True,
    keep_hs=True,
    sanitize=False,
    force_reload=False,
)

NameError: name 'MultiCMPNNDatasetSDF' is not defined

In [ ]:
ds[0][0].y

tensor([1.1457, 1.3843])

In [ ]:
train_idx, test_idx, val_idx = random_split_indices(ds)

In [ ]:
len(train_idx), len(test_idx), len(val_idx)

(1445, 181, 181)

In [ ]:
mean, std = ds.compute_normalization(train_idx)

In [ ]:
print(f"mean: {mean}, std: {std}")

mean: tensor([1.2656, 1.3868]), std: tensor([0.0821, 0.0743])


In [ ]:
ds.apply_normalization()

In [ ]:
# 6) create train/val/test splits
train_ds = Subset(ds, train_idx)
val_ds = Subset(ds, val_idx)
test_ds = Subset(ds, test_idx)

In [ ]:
from utils_paired import collate_pairs
# Use pytorch pure data loader
from torch.utils.data import DataLoader
# 7) create dataloaders
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, collate_fn=collate_pairs)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, collate_fn=collate_pairs)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, collate_fn=collate_pairs)
sample_pairs = [train_ds[i] for i in range(4)]   # 4 tuple-pairs
batch        = collate_pairs(sample_pairs)       # now returns one Batch
print(type(batch), batch.num_graphs)             # Batch 8

batch.num_graphs

<class 'abc.DataBatch'> 8


8

In [ ]:
from torchmetrics import MeanSquaredError, MeanAbsoluteError, R2Score
# Check device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from metrics import MSE, RMSE, R2Score, MAE
model = MultiCMPNNLitModel(
    in_node_feats=ds.num_atom_features,
    in_edge_feats=ds.num_bond_features,
    hidden_dim=300,
    num_steps=3,
    dropout_mp=0.1,
    dropout_head=0.1,
    n_tasks=ds.num_targets,
    metrics=[MSE(), RMSE(), R2Score(), MAE()],
    lr=1e-4,
    weight_decay=1e-3,
    target_mean=ds.mean,
    target_std=ds.std,
    mpn_shared=False
)

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="cmpnn")


trainer = Trainer(
    max_epochs=300,
    accelerator="gpu",
    devices=1,
    logger=logger,
    # Allow for early stopping
    callbacks=[
    EarlyStopping("val_mse", patience=10),
    ModelCheckpoint(monitor="val_mse", mode="min")
],
    # Log to TensorBoard

    enable_progress_bar=True,
)
trainer.fit(
    model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | encoder1  | CMPNNEncoder | 3.5 M  | train
1 | encoder2  | CMPNNEncoder | 3.5 M  | train
2 | head      | FFNHead      | 721 K  | train
3 | metrics   | ModuleDict   | 0      | train
4 | criterion | MSE          | 0      | train
---------------------------------------------------
7.8 M     Trainable params
0         Non-trainable params
7.8 M     Total params
31.034    Total estimated model params size (MB)
70        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/calvin/miniforge3/envs/dmpnn_rocm/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (23) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
trainer.test(
    model,
    dataloaders=test_loader,
    ckpt_path=None,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_mae          │    0.3511275351047516     │
│        test_mae_t0        │    0.29153504967689514    │
│        test_mae_t1        │    0.41072002053260803    │
│         test_mse          │    0.1434030383825302     │
│        test_mse_t0        │    0.10598113387823105    │
│        test_mse_t1        │    0.18082493543624878    │
│     test_mse_unscaled     │   0.001703167101368308    │
│          test_r2          │    0.6510128974914551     │
│         test_rmse         │    0.3511275351047516     │
│       test_rmse_t0        │    0.29153504967689514    │
│       test_rmse_t1        │    0.41072002053260803    │
└───────────────────────────┴───────────────────────────┘

[{'test_mse': 0.1434030383825302,
  'test_mse_unscaled': 0.001703167101368308,
  'test_mse_t0': 0.10598113387823105,
  'test_mse_t1': 0.18082493543624878,
  'test_rmse': 0.3511275351047516,
  'test_rmse_t0': 0.29153504967689514,
  'test_rmse_t1': 0.41072002053260803,
  'test_r2': 0.6510128974914551,
  'test_mae': 0.3511275351047516,
  'test_mae_t0': 0.29153504967689514,
  'test_mae_t1': 0.41072002053260803}]